# Group Name: The Big One

## Group Members: Nicholas Parker, Matthew King, Sean Sturtevant

### Dataset: Predict NHL Player Salaries

1) Ask
----

Can we accurately predict the salary of an NHL player, for the 2016-2017 season?

2) Acquire
----

Link to data and data dictionary can be found [here](https://www.kaggle.com/camnugent/predict-nhl-player-salaries#train.csv).

The dataset contains 874 records of NHL players and 151 features.

3) Process
----

In [1]:
reset -fs

In [2]:
import pandas as pd
import numpy as np
import requests
import matplotlib as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import plotly.express as px
from rfpimp import *

##### Direct Feature Engineering

In [3]:
hockey_train = pd.read_csv('./data/clean/train.csv', encoding = "ISO-8859-1")

hockey_test = pd.read_csv('./data/clean/test.csv', encoding = "ISO-8859-1")

hockey_test_y = pd.read_csv('./data/clean/test_salaries.csv', encoding = "ISO-8859-1")

In [4]:
def combine_train_and_test(train_df, test_df, test_response):
    """
    Combine the train and test datasets that were previous split at the source of the data.
    """
    test_df = pd.concat([test_df, test_response], axis = 1)
    return pd.concat([train_df, test_df],ignore_index = True, sort = False)

hockey = combine_train_and_test(hockey_train, hockey_test, hockey_test_y)

## Injecting New Data

### We want to make sure that players who were all Stars have a feature that indicates such.

In [5]:
# Below is the link for the all stars from the year 2017.
url = "https://www.hockey-reference.com/allstar/NHL_2017_roster.html"
html = requests.get(url, 'html-parser')

In [6]:
tables = pd.read_html(html.text)

In [7]:
all_stars = []
for table in tables:
    all_stars += list(table['Player'])

In [8]:
hockey['Name'] = hockey['First Name'] + ' ' + hockey['Last Name']
hockey.loc[(hockey.Name.isin(all_stars)), "All_Star"] = 1
hockey.loc[(hockey.All_Star.isna()), "All_Star"] = 0

In [9]:
def nationality_group(df, nationalityCol):
    """
    Reduces the number of values in the Nationality column through binning.
    """
    # A function to feature engineering the 'Nationality column'
    # Changes it from 16 unique values to 5 to prevent overfitting
    scandanavianNations = ['SWE','NOR','FIN']
    otherNations = ['CHE','CZE','FRA','DEU','SVK','AUT','DNK','LVA','HRV','GBR','SVN']
    df.loc[(df[nationalityCol].isin(scandanavianNations)), nationalityCol] = 'Scandanavian'
    df.loc[(df[nationalityCol].isin(otherNations)), nationalityCol] = 'Other'
    return df

hockey = nationality_group(hockey, 'Nat')

In [10]:
# Code used to group and remove provinces and states that are only seen a few times
# Useful to prevent overfitting to values only observed a few times

extreneousStates = ['AK', 'AL', 'AZ', 'CO', 'CT', 'FL', 'IN', 'ME', 'MO', 'NC'
                    , 'ND', 'NE', 'NH', 'NJ', 'NL', 'NS', 'OH', 'OK', 'PA'
                    , 'PE', 'RI', 'SC', 'TX', 'UT', 'WA']

hockey.loc[(hockey['Pr/St'].isin(extreneousStates)),'Pr/St'] = 'Other'

# Removing the time variable 'Born' by making a variable 'Age'
hockey['Age'] = 117 - pd.to_numeric(hockey['Born'].str[0:2])
hockey.loc[(hockey['Age'] > 99), 'Age'] = hockey['Age'] - 100

In [11]:
# Adding isNa Cols
# These columns are useful to account for missing data
def addIsNACol(df, col_name):
    """
    Add columns that indicate whether a record had missing data for specified features.
    """
    na_col_name = col_name + '_is_na'
    df[na_col_name] = 0
    df.loc[(df[col_name].isna()), na_col_name] = 1
    return df

hockey = addIsNACol(hockey, 'DftYr')
hockey = addIsNACol(hockey, 'iCF')

##### Save Processed Data to be used by Model Pipeline
Further work with the columns will be done in the pipeline by excluding variable and imputation

In [12]:
# hockey.to_csv('./data/processed/hockey.csv', index= False)

4) Exploratory Data Analysis
----

## How large is our dataset?

In [13]:
hockey.shape

(874, 159)

<span style='font-size:18px'> 
    Our data contains 874 observations with 154 possible features to be included into a model.<br>
    In order to use the features, we should have an understanding of which features contain missing<br>
    values, and which features do not contain any missing values.
</span>

### Columns with more than one missing value

In [14]:
missing = np.sum(hockey.isna())
missing_df = pd.DataFrame()
missing_df['Column'] = hockey.columns[missing > 0]
missing_df['Num Missing'] = missing[missing > 0].values
missing_df.sort_values('Num Missing', ascending=False).head(15)

,Column,Num Missing
0,Pr/St,225
2,DftRd,125
3,Ovrl,125
1,DftYr,125
21,sDist.1,25
13,iCF,11
24,iHA,11
18,iRB,11
15,iSF,11
14,iFF,11


# Distribution of Target Column

<span style='font-size:18px'> 
    Since we are trying to predict the salary of an NHL player, it might be useful for us to visualize<br>
    the distribution of the salary of players.
</span>

First verify that we have no missing values in our target column:

In [15]:
sum(hockey.Salary.isna())

0

In [16]:
unique, counts = np.unique(hockey.Nat, return_counts=True)
dict(zip(unique, counts))

{'CAN': 409, 'Other': 79, 'RUS': 36, 'Scandanavian': 111, 'USA': 239}

## Is there a salary difference for players who are undrafted vs. those who were drafted?

In [17]:
# Adding a column for whether or not the player is undrafted.
hockey = addIsNACol(hockey, 'DftYr')

In [18]:
drafted = hockey.loc[(hockey.DftYr_is_na == 0)][['Salary', 'DftYr_is_na']]
undrafted = hockey.loc[(hockey.DftYr_is_na == 1)][['Salary', 'DftYr_is_na']]

#### ** The code below will be commented out, because we aren't sure if rpy2 is installed on your machine.
#### We will still link to the visualization that was created.

In [19]:
# %R -i undrafted
# %R -i drafted
# %R library(ggplot2)

In [20]:
# %%R -w 8 -h 4 --units in -r 200
# ggplot() +
#     geom_density(aes(x=Salary), color="Blue", data=undrafted) + 
#     geom_density(aes(x=Salary), color="Red", data=drafted)

<img src="images/SalaryDist.png">

<span style='font-size:18px'> 
    From what we can see above, there doesn't appear to be much of a difference between players<br>
    salary depending on whether or not they were drafted or if they were undrafted.
</span>

# Is there a correlation between Age and Salary?

In [21]:
age = hockey[['Salary', 'Age']]
# %R -i age

In [22]:
# %%R -w 8 -h 4 --units in -r 200
# ggplot(aes(x=Age, y=Salary), data=age) +
#     geom_point(aes(x=Age, y=Salary), color='blue', alpha=0.7, data=age) +
#     geom_smooth(method = "lm", color="red", alpha=0.3)
# # ggsave("age.png", dpi=200)

<img src="images/age.png">

# How many different players Nationality is there?

In [23]:
hockey_clean = combine_train_and_test(hockey_train, hockey_test, hockey_test_y)

In [24]:
hockey_clean_nat = hockey_clean.groupby('Nat').agg({'Salary':'count'}).reset_index()
hockey_clean_nat['Dataset'] = "Raw"

In [25]:
hockey_nat = hockey.groupby('Nat').agg({'Salary':'count'}).reset_index()
hockey_nat.loc[(hockey_nat.Nat == "Other"), "Nat"] = "Other Countries"
hockey_nat['Dataset'] = "Processed"

In [26]:
# %R -i hockey_nat
# %R -i hockey_clean_nat

In [27]:
# %%R -w 8 -h 6 --units in -r 200
# ggplot() +
#     geom_bar(aes(x=reorder(Nat, Salary), y=Salary), stat="identity", data=hockey_clean_nat) +
#     ylab("Number of Players\n") +
#     xlab("Country\n") +
#     coord_flip() +
#     theme(axis.ticks.y=element_blank(), axis.text.y=element_text(size=12, color="black"), axis.title.y=element_text(size=14),
#           axis.title.x=element_text(size=14), axis.ticks.x=element_blank(), axis.text.x=element_text(size=12,color="black"))
# # ggsave("images/country1.png", dpi=200)

<img src="images/country1.png">

<span style='font-size:18px'> 
    After looking at the number of players by each country, we noticed that many of the players <br>
    primarily from four different countries. There were also many different European countries that<br>
    that we thought would be appropriate to group together since how few the number of counts that<br>
    there was. Below, you can see the number of groups that we ended up grouping Nationality into.<br><br>
</span>

### After processing the data, how many Nationality categories to we have?

In [28]:
unique, counts = np.unique(hockey.Nat, return_counts=True)
dict(zip(unique, counts))

{'CAN': 409, 'Other': 79, 'RUS': 36, 'Scandanavian': 111, 'USA': 239}

In [29]:
# %%R -w 8 -h 6 --units in -r 200
# ggplot() +
#     geom_bar(aes(x=reorder(Nat, Salary), y=Salary), stat="identity", data=hockey_nat) +
#     ylab("Number of Players\n") +
#     xlab("Country\n") +
#     coord_flip() +
#     theme(axis.ticks.y=element_blank(), axis.text.y=element_text(size=12, color="black"), axis.title.y=element_text(size=14),
#           axis.title.x=element_text(size=14), axis.ticks.x=element_blank(), axis.text.x=element_text(size=12,color="black"))
# # ggsave("images/country2.png", dpi=200)

<img src="images/country2.png">

5) Models
----

In [30]:
def add_commas(number):
    """
    Adds commas to values greater that 1,000 for evaluation metrics.
    """
    return ("{:,}".format(number))

In [31]:
def mape_metric(y_test, y_pred):
    """
    Calculates the mean absolute percentage error.
    """
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean((np.abs(y_test - y_pred) / (0.5*(np.abs(y_test)+np.abs(y_pred)))))*100

In [32]:
# Create train, test, and validation sets of the data.
y = hockey['Salary']
X = hockey.drop('Salary', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=99) # Split to obtain the test set
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, random_state=99) # Split to obtain the validation set

### Baseline Model

In [33]:
# Create separate lists of numeric and categorical features to be passed through the pipeline.
numeric_baseline = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number)]
categorical_baseline = [feature for feature in X_train.columns if feature not in numeric_baseline]

In [34]:
def make_pipeline_ridge(regressor=None):
    """
    Creates pipeline to perform transformations on numeric and categorical features 
    and pass into a Ridge Regression Model.
    """
    numeric_features = numeric_baseline
    numeric_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='median')),
        ('scaler', preprocessing.StandardScaler())])

    categorical_features = categorical_baseline
    categorical_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = compose.ColumnTransformer(transformers=[
        ('numerical', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', regressor)])
    
    return pipeline

regressor = linear_model.Ridge(alpha=100, tol=0.001)
pipeline = make_pipeline_ridge(regressor)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                       

##### Baseline Evaluation

In [35]:
mean_squared_error_scorer = make_scorer(metrics.mean_squared_error)
cross_val_bl = np.sqrt(cross_val_score(pipeline, 
                                       X_train, 
                                       y_train, 
                                       scoring=mean_squared_error_scorer,
                                       cv=10))
print(cross_val_bl)
print('')
print(f"{add_commas(round(cross_val_bl.mean(), 2))} average cross validation rmse")

[1453721.08403749 1303208.72307592 1302488.44918904 1700854.91509345
 1795888.74975077 1762539.65712339 1334242.46954355 1455163.9307866
 1497014.57788712 1171192.24732952]

1,477,631.48 average cross validation rmse


Root Mean Squared Error

In [36]:
y_pred = pipeline.predict(X_train)
rmse_value_train = add_commas(round(np.sqrt(metrics.mean_squared_error(y_train, y_pred)), 2))
print(f"{rmse_value_train} rmse on train dataset")

y_pred = pipeline.predict(X_validation)
rmse_value_validation = add_commas(round(np.sqrt(metrics.mean_squared_error(y_validation, y_pred)), 2))
RidgeRMSE = round(np.sqrt(metrics.mean_squared_error(y_validation, y_pred)), 2)
print(f"{rmse_value_validation} rmse on validation dataset")

1,182,093.04 rmse on train dataset
1,534,370.47 rmse on validation dataset


Median Absolute Error

In [37]:
y_pred = pipeline.predict(X_train)
medae_value_train = add_commas(round(metrics.median_absolute_error(y_train, y_pred), 2))
print(f"${medae_value_train} medae on train dataset")

y_pred = pipeline.predict(X_validation)
medae_value_validation = add_commas(round(metrics.median_absolute_error(y_validation, y_pred), 2))
RidgeMedAE = medae_value_validation
print(f"${medae_value_validation} medae on validation dataset")

$620,274.51 medae on train dataset
$731,403.67 medae on validation dataset


Mean Absolute Percentage Error

In [38]:
y_pred = pipeline.predict(X_train)
mape_value_train = round(mape_metric(y_train, y_pred), 2)
print(f"{mape_value_train}% sMAPE on train dataset")

y_pred = pipeline.predict(X_validation)
mape_value_validation = round(mape_metric(y_validation, y_pred), 2)
RidgesMAPE = mape_value_validation
print(f"{mape_value_validation}% sMAPE on validation dataset")

48.43% sMAPE on train dataset
57.93% sMAPE on validation dataset


### Model Selection Process

### knn Model

In [39]:
nonpredictive_features = ['ENG', 'Wide', 'Over', 'PSG', 'PSA', 'S.Dflct', 'G.Bkhd', 'Post', 'G.Dflct', 'CBar ', 'G.Slap', 'G.Snap', 'G.Wrst', 'G.Wrap', 'G.Tip', 'S.Bkhd', 'Min', 'S.Slap', 'Misc', 'Noise', 'DAP', 'Grit', 'PS', 'DPS', 'OPS', 'DSA', 'DSF', 'Game', 'Match', 'S.Snap', 'Maj', '1G', 'NPD', 'iPenDf', 'iPenD', 'iPenT', 'S.Wrst', 'S.Wrap', 'S.Tip', 'GWG', 'FOL.Down', 'OTG', 'PIM', 'iSF.1', 'iCF.1', 'Diff', 'Pct%', 'FOL.Close', 'TOI/GP.1', 'TOI/GP', 'TOI', 'Shifts', 'E+/-', 'sDist', '+/-', 'PTS', 'A2', 'A1', 'A', 'G', 'GP', 'Wt', 'Ht', 'iSF.2', 'Age', 'iFOW', 'iBLK', 'iFOL', 'dzFOL', 'nzFOW', 'nzFOL', 'ozFOW', 'ozFOL', 'dzFOW', 'FOL.Up', 'FOW.Up', 'iTKA', 'iGVA', 'iMiss', 'FOW.Down', 'iHF', 'FOW.Close', 'FO%', 'Position', 'Team', 'PEND', 'TOIX', 'GA', 'xGA', 'iSCF', 'iPEND', 'sDist.1', 'iHF.1', 'PDO', 'Hand', 'SCA', 'iTKA.1', 'SA', 'IPP%', 'ixG', 'FA', 'Pace', 'iGVA.1', 'SV%', 'RBF', 'PENT', 'F/60', 'GVA', 'TKA', 'FOW', 'Diff/60']

# For KNN, we only want to include the numeric variables.
# At the moment we do not want to consider some of the categorical 
# variables since KNN will not handle these naturally.
numeric_knn = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number) 
                      and feature not in nonpredictive_features]

In order to be able to fit the knn model, we will need to make sure that our pipeline is suited to handle the data for KNN.

In [40]:
def make_pipeline_knn(knn=None):
    """
    Creates pipeline that performs separate transformations on the categorical and numerical features
    for a KNN algorithm.
    """
    
    # All predictive numeric features
    numeric_features = numeric_knn
    numeric_transformer = Pipeline(steps=[
        # Impute any missing values with the median.
        ('imputer', impute.SimpleImputer(strategy='median')),
        # Make sure that all values are Normalized for KNN. 
        # Since we are going to look at points that are similar in
        # terms of euclidean space, we need all features on the same
        # scale.
        ('normalizer', preprocessing.Normalizer())])

    preprocessor = compose.ColumnTransformer(transformers=[
        ('numerical', numeric_transformer, numeric_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('knn', knn)])
    
    return pipeline

knn = KNeighborsRegressor()
pipeline_knn = make_pipeline_knn(knn)

In [41]:
def make_random_cv_knn():
    """
    Define hyperparameter search space for KNN algorithm
    Instantiate RandomizedSearchCV with the pipeline.
    """
    
    algo = ['ball_tree', 'kd_tree', 'auto']
    weights = ['distance', 'uniform']
    neighbors = [8, 9, 10, 15]
    hyperparameters = dict(knn__algorithm=algo,
                          knn__n_neighbors=neighbors,
                          knn__weights=weights)
    
    reg_random_cv = RandomizedSearchCV(pipeline_knn, 
                                       hyperparameters, 
                                       cv=5, 
                                       n_iter=15, 
                                       verbose=1,
                                       random_state=42,
                                       n_jobs=-1)
    
    return reg_random_cv

model_knn = make_random_cv_knn()
model_knn.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:    2.9s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('preprocessor',
                                              ColumnTransformer(n_jobs=None,
                                                                remainder='drop',
                                                                sparse_threshold=0.3,
                                                                transformer_weights=None,
                                                                transformers=[('numerical',
                                                                               Pipeline(memory=None,
                                                                                        steps=[('imputer',
                                                                                                SimpleImputer(add_indicator=False,
                                                               

In [42]:
mean_squared_error_scorer = make_scorer(metrics.mean_squared_error)
cross_val_knn = np.sqrt(cross_val_score(model_knn.best_estimator_, 
                                       X_train, 
                                       y_train, 
                                       scoring=mean_squared_error_scorer,
                                       cv=10))
print(cross_val_knn)
print('')
print(f"{add_commas(round(cross_val_knn.mean(), 2))} average cross validation rmse")

[1632972.63524097 1673164.81004026 1595420.0474987  1861409.78294072
 1967382.04380449 2067575.95721091 1642673.46192757 1675774.57859102
 1589030.16179889 1150602.21324031]

1,685,600.57 average cross validation rmse


Root Mean Squared Error

In [43]:
knn_y_pred = model_knn.best_estimator_.predict(X_train)
rmse_value_train = add_commas(round(np.sqrt(metrics.mean_squared_error(y_train, knn_y_pred)), 2))
print(f"{rmse_value_train} rmse on train dataset")

knn_y_pred = model_knn.best_estimator_.predict(X_validation)
rmse_value_validation = add_commas(round(np.sqrt(metrics.mean_squared_error(y_validation, knn_y_pred)), 2))
KNNRMSE = round(np.sqrt(metrics.mean_squared_error(y_validation, knn_y_pred)), 2)
print(f"{rmse_value_validation} rmse on validation dataset")

1,539,761.45 rmse on train dataset
1,649,920.01 rmse on validation dataset


Median Absolute Error

In [44]:
knn_y_pred = model_knn.best_estimator_.predict(X_train)
medae_value_train = add_commas(round(metrics.median_absolute_error(y_train, knn_y_pred), 2))
print(f"${medae_value_train} medae on train dataset")

knn_y_pred = model_knn.best_estimator_.predict(X_validation)
medae_value_validation = add_commas(round(metrics.median_absolute_error(y_validation, knn_y_pred), 2))
KNNRMedAE = medae_value_validation
print(f"${medae_value_validation:} medae on validation dataset")

$490,833.33 medae on train dataset
$540,881.94 medae on validation dataset


Mean Absolute Percentage Error

In [45]:
knn_y_pred = model_knn.best_estimator_.predict(X_train)
mape_value_train = round(mape_metric(y_train, knn_y_pred), 2)
print(f"{mape_value_train}% sMAPE on train dataset")

knn_y_pred = model_knn.best_estimator_.predict(X_validation)
mape_value_validation = round(mape_metric(y_validation, knn_y_pred), 2)
KNNsMAPE = mape_value_validation
print(f"{mape_value_validation}% sMAPE on validation dataset")

40.87% sMAPE on train dataset
43.57% sMAPE on validation dataset


##### Random Forest Model

In [46]:
# These are features which should have no affect on Salary, like First Name
illogical_features = ['Born', 'City', 'Last Name', 'First Name', 'Name', 'Cntry']
# These are features which are repeated later on with updated stats
redundant_features = ['TOI/GP', 'iCF', 'iSF', 'iSF.1', 'sDist', 'iHF', 'iGVA', 'iTKA', 'iBLK', 'iFOW', 'iFOL']
# These are features found to be non-predictive using Parrt's rfpimp package

drop_list = illogical_features + redundant_features

numeric_rf = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number) 
                      and feature not in drop_list]
categorical_rf = [feature for feature in X_train.columns if feature not in numeric_rf
                       and feature not in drop_list]

In [47]:
def make_pipeline_rf(regressor=None):
    """
    Creates pipeline that performs separate transformations on the categorical and numerical features.
    """
    
    numeric_features = numeric_rf
    numeric_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='median'))])

    categorical_features = categorical_rf
    categorical_transformer = Pipeline(steps=[
        ('imputer', impute.SimpleImputer(strategy='constant', fill_value='unknown')),
        ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = compose.ColumnTransformer(transformers=[
        ('numerical', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)])

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', regressor)])
    
    return pipeline

regressor_rf = RandomForestRegressor()
pipeline_rf = make_pipeline_rf(regressor_rf)

In [ ]:
def make_random_cv():
    """
    Define hyperparameter search space
    Instantiate RandomizedSearchCV with the pipeline.
    """
    
    bootstrap = ['True', 'False']
    oob_score = ['True', 'False']
    max_features = ['auto', 'sqrt']
    min_samples_leaf = [2, 3, 5, 6 , 7, 8, 9, 10]
    n_estimators = [100, 150, 200, 300, 500]
    hyperparameters = dict(regressor__min_samples_leaf=min_samples_leaf,
                          regressor__bootstrap=bootstrap,
                          regressor__max_features=max_features,
                          regressor__n_estimators=n_estimators,
                          regressor__oob_score=oob_score)
    reg_random_cv = RandomizedSearchCV(pipeline_rf, 
                                       hyperparameters, 
                                       cv=5, 
                                       n_iter=15, 
                                       verbose=1,
                                       random_state=42,
                                       n_jobs=-1)
    
    return reg_random_cv

model_rf = make_random_cv()
model_rf.fit(X_train, y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Cross Validation Scores for Best Estimator from Randomized Search

In [ ]:
mean_squared_error_scorer = make_scorer(metrics.mean_squared_error)
cross_val_rf = np.sqrt(cross_val_score(model_rf.best_estimator_, 
                                       X_train, 
                                       y_train, 
                                       scoring=mean_squared_error_scorer,
                                       cv=10))
print(cross_val_rf)
print('')
print(f"{add_commas(round(cross_val_rf.mean(), 2))} average cross validation rmse")

Root Mean Squared Error

In [ ]:
y_pred = model_rf.predict(X_train)
rmse_value_train = add_commas(round(np.sqrt(metrics.mean_squared_error(y_train, y_pred)), 2))
print(f"{rmse_value_train} rmse on train dataset")

y_pred = model_rf.predict(X_validation)
rmse_value_validation = add_commas(round(np.sqrt(metrics.mean_squared_error(y_validation, y_pred)), 2))
RFRMSE = round(np.sqrt(metrics.mean_squared_error(y_validation, y_pred)), 2)
print(f"{rmse_value_validation} rmse on validation dataset")

Median Absolute Error

In [ ]:
y_pred = model_rf.predict(X_train)
medae_value_train = add_commas(round(metrics.median_absolute_error(y_train, y_pred), 2))
print(f"${medae_value_train} medae on train dataset")

y_pred = model_rf.predict(X_validation)
medae_value_validation = add_commas(round(metrics.median_absolute_error(y_validation, y_pred), 2))
RFMedAE = medae_value_validation
print(f"${medae_value_validation} medae on validation dataset")

Mean Absolute Percentage Error

In [ ]:
y_pred = model_rf.predict(X_train)
mape_value_train = round(mape_metric(y_train, y_pred), 2)
print(f"{mape_value_train}% sMAPE on train dataset")

y_pred = model_rf.predict(X_validation)
mape_value_validation = round(mape_metric(y_validation, y_pred), 2)
RFsMAPE = mape_value_validation
print(f"{mape_value_validation}% sMAPE on validation dataset")

## Final Model Choice

### Model Performance Visualization

In [ ]:
def plotWithPlotly(performance_df):
    fig = px.bar(performance_df, x = 'Models', y = 'RMSE'
            , color = 'RMSE'
            , hover_data= ["MedAE",'sMAPE']
            )
    fig.show()
    
def plotWithMatplotlib(performance_df):
    performance_df[['Models','RMSE']].plot(kind = 'bar', x = 'Models', y = 'RMSE')

In [ ]:
model_performance_df = pd.DataFrame()
model_performance_df['Models'] = ['Ridge','Random Forest','KNN']
model_performance_df['RMSE'] = [RidgeRMSE, RFRMSE, KNNRMSE]
model_performance_df['MedAE'] = [RidgeMedAE, RFMedAE, KNNRMedAE]
model_performance_df['sMAPE'] = [RidgesMAPE, RFsMAPE, KNNsMAPE]
model_performance_df['RMSE'] = pd.to_numeric(model_performance_df['RMSE'])

In [ ]:
try:
    plotWithPlotly(model_performance_df)
except:
    plotWithMatplotlib(model_performance_df)

Using RMSE as our North Star evaluation metric to choose a model, we ended up choosing to continue with the Random Forest Model because it consistently had the best validation set test scores.

Feature Importance Analysis

In [ ]:
# Using the rfpimp package by ParrT, we look at our best model and estimate feature importance
I = importances(model_rf.best_estimator_, X_train, y_train)
I.reset_index(inplace = True)

# These features deemed 'not important are dropped'
# It doesn't affect the training score much, but a simpler model generalizes better
print(len(list(I.loc[(I.Importance <= 0)]['Feature'])), 'Features')
new_drop_list = list(I.loc[(I.Importance <= 0)]['Feature'])

In [ ]:
# Retrain the model on the new drop list
numeric_rf = [feature for feature in X_train.columns if np.issubdtype(X_train[feature], np.number) 
                      and feature not in new_drop_list]
categorical_rf = [feature for feature in X_train.columns if feature not in numeric_rf
                       and feature not in new_drop_list]

regressor_rf = RandomForestRegressor()
pipeline_rf = make_pipeline_rf(regressor_rf)
model_rf = make_random_cv()
model_rf.fit(X_train, y_train)

6) Deliver
----

### Evaluation Metrics

##### Median Absolute Error Test Set

In [ ]:
y_pred = model_rf.predict(X_test)
medae_value_test = add_commas(round(metrics.median_absolute_error(y_test, y_pred), 2))
print(f"${medae_value_test:} medae on test dataset")

##### Root Mean Squared Error Test Set

In [ ]:
y_pred = model_rf.predict(X_test)
rmse_value_test = add_commas(round(np.sqrt(metrics.mean_squared_error(y_test, y_pred)), 2))
print(f"{rmse_value_test} rmse on test dataset")

##### Mean Absolute Percentage Error Test Set

In [ ]:
y_pred = model_rf.predict(X_test)
mape_value_test = round(mape_metric(y_test, y_pred), 2)
print(f"{mape_value_test}% sMAPE on test dataset")

### Summary and Takeaways

<span style='font-size:18px'> 
To predict NHL player salaries, we fitted three regressor models, a Ridge Regression model, a KNN model, and a Random Forest model. After running our models against each other, we looked at three different evaluation metrics for each. Each evaluation metric served a unique purpose for being included throughout our process. Median Absolute Error (MedAE) is the most useful for external communication of our model as it is the most interpretable. Being able to discuss an evaluation metric in terms of dollars allows for those without machine learning experience to understand exactly what that metric is telling us about the error in our model. Symmetric Mean Absolute Percent Error (sMAPE) is useful to us because it provides a completely different interpretation. Since the values we are predicting are so large it is likely that the errors may also be large numbers, so being able to interpret the error in relation to the actual values is useful. For example, a median absolute error of hundreds of thousands of dollars seems like it is abnormally large, but in comparison to the actual y-values this may be a fairly decent prediction error. Finally, our 'North Star' metric that we use to pick a model is the Root Mean Square Error (RMSE). RMSE minimizes the loss, and unlike MAPE, RMSE is symmetric. MAPE sufferes from asymmetry.</span>

<span style='font-size:18px'> 
Our single best model was our Random Forest model which had the lowest of all three evaluation metrics on the validation sets, included an RMSE of 1.4 million. We then ran this model on the test set. To interpret the test set results, we can use the sMAPE of 38\% and \\$540,125 on the test set. A 38\% sMAPE score tells us that the average error was 38\% of the actual salary and a MedAE score of value of \\$540,125 tells us that the median error from our model is \\$540,125 off from the actual salary. Even with such large salary values this error is quite large and we do not necessarily believe this model should be pushed into production to any NHL General Managers to give them a way to predict salary in the ever-changing salary cap environment of the NHL.</span>
